# 교보문고 리뷰 크롤링

## 절차
1. '베스트셀러_통합.csv'에서 ISBN 리스트 추출
2. 링크 진입 후 ISBN별 도서 상세페이지 진입
3. 리뷰 탭 클릭
4. 수집(작성자, 본문, 별점)

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("베스트셀러_통합.csv", encoding='utf-8-sig')  # 또는 'utf-8'

# 특정 열의 값들을 리스트로 추출
isbn_list = df["ISBN"].tolist()

# 결과 출력 (선택사항)
print(isbn_list)

[9788932022888, 9788936472016, 9788983711137, 9788936471187, 9788937460197, 9788956605418, 9788983945068, 9788932020006, 9788956604992, 9788991428072, 9788954617383, 9788937886966, 9788936456023, 9788984314238, 9788954611152, 9788956605227, 9788936456221, 9788990620446, 9788972756125, 9788936452049, 9788991075535, 9788996094050, 9788938814623, 9788936486921, 9788935703081, 9788972756194, 9788991204478, 9788992647595, 9788991998452, 9788937425363, 9788965960072, 9788963306094, 9788952762054, 9788989824565, 9788992759113, 9788954616515, 9788993497014, 9788958201762, 9788959956494, 9788935661473, 9788936456085, 9788932018508, 9788970128337, 9788986836240, 9788995823231, 9788996851516, 9788932909349, 9788970126746, 9788958071853, 9788932909707, 9788954605816, 9788936433598, 9788972754824, 9788971992951, 9788958283508, 9788972754015, 9788958560760, 9788954608916, 9788954613729, 9788938201065, 9788937462672, 9788958073642, 9788956605593, 9788954617246, 9788938201010, 9791189467678, 979119163

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import re

def setup_driver():
    """웹드라이버 설정"""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def get_star_rating(block):
    """별점 추출 함수"""
    try:
        filled_stars = block.find_element(By.CSS_SELECTOR, 'span.filled-stars')
        width_style = filled_stars.get_attribute('style')
        width_match = re.search(r'width:\s*(\d+(?:\.\d+)?)%', width_style)
        if width_match:
            width_percent = float(width_match.group(1))
            # 100% = 5점, 80% = 4점, 60% = 3점, 40% = 2점, 20% = 1점
            rating = round(width_percent / 20, 1)
            return rating
        return None
    except:
        return None

def crawl_reviews_for_book(driver, isbn, max_pages=100):
    """특정 ISBN의 리뷰를 크롤링"""
    # 검색 URL로 이동 (ISBN으로 검색)
    search_url = f'https://search.kyobobook.co.kr/search?keyword={isbn}'
    
    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 10)
        time.sleep(3)
        
        # 검색 결과에서 첫 번째 도서 이미지 클릭하여 상세 페이지로 이동
        try:
            book_image = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="shopData_list"]/ul/li/div[1]/div[1]/a/span/img')))
            driver.execute_script("arguments[0].click();", book_image)
            time.sleep(3)
            print(f"ISBN {isbn}: 도서 상세 페이지로 이동 완료")
        except Exception as e:
            print(f"ISBN {isbn}: 도서 이미지 클릭 실패 - {str(e)}")
            return []
        
        # 새로운 대기 객체 생성 (페이지가 변경되었으므로)
        wait = WebDriverWait(driver, 10)
        
        # 리뷰탭 클릭 (여러 가능한 선택자로 시도)
        review_tab_selectors = [
            '//*[@id="contents"]/div[1]/div/div[2]/div[2]/div[1]/div/div[4]/button',
            '//button[contains(text(), "리뷰")]',
            '//a[contains(text(), "리뷰")]'
        ]
        
        review_tab_clicked = False
        for selector in review_tab_selectors:
            try:
                review_tab = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                review_tab.click()
                review_tab_clicked = True
                time.sleep(2)
                break
            except:
                continue
        
        if not review_tab_clicked:
            print(f"ISBN {isbn}: 리뷰 탭을 찾을 수 없습니다.")
            return []
        
        # 구매자 리뷰 탭 클릭 (여러 선택자로 시도)
        buyer_review_selectors = [
            '//*[@id="contents"]/div[2]/div[1]/div/div[1]/ul/li[3]/a',
            '//a[contains(text(), "구매자")]',
            '//li[contains(@class, "buyer")]/a'
        ]
        
        for selector in buyer_review_selectors:
            try:
                buyer_review_tab = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                buyer_review_tab.click()
                time.sleep(2)
                break
            except:
                continue
        
        reviews = []
        page = 1
        consecutive_empty_pages = 0
        
        while page <= max_pages and consecutive_empty_pages < 3:
            time.sleep(2)
            
            # 리뷰 블록 찾기
            review_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.comment_item')
            
            if not review_blocks:
                consecutive_empty_pages += 1
                print(f"ISBN {isbn}: {page}페이지 - 리뷰가 없습니다. (연속 빈 페이지: {consecutive_empty_pages})")
            else:
                consecutive_empty_pages = 0
                page_reviews = []
                
                for block in review_blocks:
                    try:
                        # 사용자 ID
                        user_id = block.find_element(By.CSS_SELECTOR, 'span.info_item').text
                    except:
                        user_id = ""
                    
                    try:
                        # 리뷰 내용
                        review_text = block.find_element(By.CSS_SELECTOR, 'div.comment_text').text
                    except:
                        review_text = ""
                    
                    # 별점
                    rating = get_star_rating(block)
                    
                    page_reviews.append({
                        'isbn': isbn,
                        'user_id': user_id,
                        'review': review_text,
                        'rating': rating,
                        'page': page
                    })
                
                reviews.extend(page_reviews)
                print(f"ISBN {isbn}: {page}페이지 - {len(page_reviews)}개 리뷰 수집 (누적: {len(reviews)}개)")
            
            # 다음 페이지 버튼 찾기 및 클릭
            try:
                # 다음 페이지 버튼 선택자들
                next_btn_selectors = [
                    'button.btn_page.next',
                    '//*[@id="ReviewList1"]/div[3]/div[2]/div/div[2]/button[2]',
                    '//button[contains(@class, "next")]',
                    '//button[contains(@class, "btn_page") and contains(@class, "next")]'
                ]
                
                next_btn = None
                for selector in next_btn_selectors:
                    try:
                        if selector.startswith('//') or selector.startswith('//*'):
                            next_btn = driver.find_element(By.XPATH, selector)
                        else:
                            next_btn = driver.find_element(By.CSS_SELECTOR, selector)
                        break
                    except:
                        continue
                
                if next_btn is None:
                    print(f"ISBN {isbn}: 다음 페이지 버튼을 찾을 수 없습니다.")
                    break
                
                # 버튼이 비활성화되어 있는지 확인
                if "disabled" in next_btn.get_attribute("class") or not next_btn.is_enabled():
                    print(f"ISBN {isbn}: 마지막 페이지입니다.")
                    break
                
                # 버튼 클릭
                driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_btn)
                
                # 페이지 로딩 대기
                time.sleep(3)
                
            except Exception as e:
                print(f"ISBN {isbn}: 다음 페이지 이동 중 오류 - {str(e)}")
                break
            
            page += 1
        
        print(f"ISBN {isbn}: 총 {len(reviews)}개 리뷰 수집 완료")
        return reviews
        
    except Exception as e:
        print(f"ISBN {isbn}: 크롤링 중 오류 발생 - {str(e)}")
        return []

def main():
    """메인 실행 함수"""
    # ISBN 리스트 (여기에 크롤링할 ISBN들을 추가)
    isbn_list = [
        "9791168270749", "9788901294742",
        # "9788960517813",  # 예시 ISBN 추가
        # "9788936434267",  # 예시 ISBN 추가
        # 더 많은 ISBN을 여기에 추가
    ]
    
    driver = setup_driver()
    all_reviews = []
    
    try:
        for i, isbn in enumerate(isbn_list, 1):
            print(f"\n=== {i}/{len(isbn_list)} - ISBN: {isbn} 크롤링 시작 ===")
            
            reviews = crawl_reviews_for_book(driver, isbn)
            all_reviews.extend(reviews)
            
            print(f"ISBN {isbn} 완료. 현재까지 총 {len(all_reviews)}개 리뷰 수집")
            
            # 각 ISBN 처리 후 잠시 대기
            time.sleep(5)
        
        # 모든 리뷰를 하나의 DataFrame으로 합치기
        if all_reviews:
            df = pd.DataFrame(all_reviews)
            
            # 결과 저장
            output_filename = f'book_reviews_all_{len(isbn_list)}books.csv'
            df.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"\n=== 크롤링 완료 ===")
            print(f"총 도서 수: {len(isbn_list)}")
            print(f"총 리뷰 수: {len(df)}")
            print(f"파일명: {output_filename}")
            print(f"\n도서별 리뷰 수:")
            print(df.groupby('isbn').size().to_string())
            
            # 별점 통계
            if df['rating'].notna().sum() > 0:
                print(f"\n별점 통계:")
                print(f"평균 별점: {df['rating'].mean():.2f}")
                print(f"별점 분포:")
                print(df['rating'].value_counts().sort_index().to_string())
            
        else:
            print("수집된 리뷰가 없습니다.")
            
    except KeyboardInterrupt:
        print("\n사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"오류 발생: {str(e)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

### 누락된 리뷰들 다시 크롤링하기

In [ ]:
import pandas as pd
kb_review_best = pd.read_csv('[교보] 베스트셀러 리뷰.csv')
no_review_list = kb_review_best[kb_review_best['review'].isna()]['isbn'].dropna().astype(str).tolist()

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import re

def setup_driver():
    """웹드라이버 설정"""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def get_star_rating(block):
    """별점 추출 함수 - 새로운 HTML 구조에 맞게 수정"""
    try:
        # 새로운 별점 구조에 맞게 수정
        filled_stars = block.find_element(By.CSS_SELECTOR, 'span.filled-stars')
        width_style = filled_stars.get_attribute('style')
        width_match = re.search(r'width:\s*(\d+(?:\.\d+)?)%', width_style)
        if width_match:
            width_percent = float(width_match.group(1))
            # 100% = 5점, 80% = 4점, 60% = 3점, 40% = 2점, 20% = 1점
            rating = round(width_percent / 20, 1)
            return rating
        return None
    except:
        # 대안적인 별점 추출 방법
        try:
            rating_element = block.find_element(By.CSS_SELECTOR, 'div.rating-container span')
            rating_text = rating_element.get_attribute('title')
            if rating_text and 'Star' in rating_text:
                # "One Star", "Two Stars" 등에서 숫자 추출
                if 'One' in rating_text:
                    return 1.0
                elif 'Two' in rating_text:
                    return 2.0
                elif 'Three' in rating_text:
                    return 3.0
                elif 'Four' in rating_text:
                    return 4.0
                elif 'Five' in rating_text:
                    return 5.0
        except:
            pass
        return None

def crawl_reviews_for_book(driver, isbn, max_pages=100):
    """특정 ISBN의 리뷰를 크롤링"""
    # 검색 URL로 이동 (ISBN으로 검색)
    search_url = f'https://search.kyobobook.co.kr/search?keyword={isbn}'
    
    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 15)  # 대기 시간 증가
        time.sleep(3)
        
        # 검색 결과에서 첫 번째 도서 이미지 클릭하여 상세 페이지로 이동
        try:
            # 여러 가능한 선택자로 시도
            book_selectors = [
                '//*[@id="shopData_list"]/ul/li/div[1]/div[1]/a/span/img',  # 기존
                '//*[@id="shopData_list"]/ul/li[1]//img',  # 더 일반적인 형태
                '//div[@id="shopData_list"]//li[1]//img',  # XPath 방식
                '.shopData_list .prod_item:first-child img'  # CSS 선택자
            ]
            
            book_clicked = False
            for selector in book_selectors:
                try:
                    if selector.startswith('//') or selector.startswith('//*'):
                        book_element = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                    else:
                        book_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
                    
                    driver.execute_script("arguments[0].click();", book_element)
                    book_clicked = True
                    time.sleep(3)
                    break
                except:
                    continue
            
            if not book_clicked:
                print(f"ISBN {isbn}: 도서를 찾을 수 없습니다.")
                return []
                
            print(f"ISBN {isbn}: 도서 상세 페이지로 이동 완료")
        except Exception as e:
            print(f"ISBN {isbn}: 도서 이미지 클릭 실패 - {str(e)}")
            return []
        
        # 새로운 대기 객체 생성 (페이지가 변경되었으므로)
        wait = WebDriverWait(driver, 15)
        
        # 리뷰탭 클릭 - 제공된 정보에 맞게 수정
        review_tab_selectors = [
            '//*[@id="contents"]/div[2]/div[1]/div/div[1]/ul/li[2]/a/span',  # 제공된 정보
            '//*[@id="contents"]/div[2]/div[1]/div/div[1]/ul/li[2]/a',
            '//button[contains(text(), "리뷰")]',
            '//a[contains(text(), "리뷰")]',
            '//*[@id="contents"]//ul/li[2]/a'  # 더 일반적인 형태
        ]
        
        review_tab_clicked = False
        for selector in review_tab_selectors:
            try:
                review_tab = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", review_tab)
                review_tab_clicked = True
                time.sleep(3)
                print(f"ISBN {isbn}: 리뷰 탭 클릭 성공")
                break
            except Exception as e:
                print(f"ISBN {isbn}: 리뷰 탭 선택자 실패 - {selector}: {str(e)}")
                continue
        
        if not review_tab_clicked:
            print(f"ISBN {isbn}: 리뷰 탭을 찾을 수 없습니다.")
            return []
        
        # 리뷰 영역이 로딩될 때까지 잠시 대기
        time.sleep(3)
        
        reviews = []
        page = 1
        consecutive_empty_pages = 0
        
        while page <= max_pages and consecutive_empty_pages < 3:
            time.sleep(3)
            
            # 리뷰 블록 찾기 - 여러 선택자로 시도
            review_block_selectors = [
                'div.comment_item',  # 기존
                'div.comment_list > div',  # 리뷰 전체를 감싸는 태그 하위
                '.review_item',
                '.comment_wrap',
                'div[class*="comment"]',
                'div[class*="review"]'
            ]
            
            review_blocks = []
            for selector in review_block_selectors:
                try:
                    review_blocks = driver.find_elements(By.CSS_SELECTOR, selector)
                    if review_blocks:
                        print(f"ISBN {isbn}: {page}페이지 - {selector}로 {len(review_blocks)}개 리뷰 블록 발견")
                        break
                except:
                    continue
            
            # 리뷰 블록을 찾은 후 전체 리뷰 로딩 탭 클릭 (첫 번째 페이지에서만)
            if page == 1 and review_blocks:
                print(f"ISBN {isbn}: 전체 리뷰 로딩 탭 클릭 시도")
                total_review_selectors = [
                    '//*[@id="ReviewList1"]/div[3]/div[1]/ul/li[1]/button/span',  # 제공된 정보
                    '//*[@id="ReviewList1"]/div[3]/div[1]/ul/li[1]/button',
                    '//button[contains(text(), "전체")]',
                    '//li[1]/button',
                    '//*[@id="ReviewList1"]//li[1]/button'
                ]
                
                total_tab_clicked = False
                for selector in total_review_selectors:
                    try:
                        total_review_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, selector)))
                        driver.execute_script("arguments[0].scrollIntoView(true);", total_review_tab)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", total_review_tab)
                        time.sleep(3)
                        print(f"ISBN {isbn}: 전체 리뷰 로딩 탭 클릭 성공")
                        total_tab_clicked = True
                        
                        # 전체 리뷰 로딩 후 다시 리뷰 블록 찾기
                        time.sleep(2)
                        for selector in review_block_selectors:
                            try:
                                review_blocks = driver.find_elements(By.CSS_SELECTOR, selector)
                                if review_blocks:
                                    print(f"ISBN {isbn}: 전체 리뷰 로딩 후 {len(review_blocks)}개 리뷰 블록 발견")
                                    break
                            except:
                                continue
                        break
                    except Exception as e:
                        print(f"ISBN {isbn}: 전체 리뷰 탭 선택자 실패 - {selector}: {str(e)}")
                        continue
                
                if not total_tab_clicked:
                    print(f"ISBN {isbn}: 전체 리뷰 로딩 탭을 찾을 수 없습니다. 기존 리뷰로 진행합니다.")
            
            if not review_blocks:
                consecutive_empty_pages += 1
                print(f"ISBN {isbn}: {page}페이지 - 리뷰가 없습니다. (연속 빈 페이지: {consecutive_empty_pages})")
                
                # 페이지에 리뷰가 없어도 다음 페이지로 이동 시도
                if consecutive_empty_pages < 3:
                    if not go_to_next_page(driver, page):
                        break
                    page += 1
                    continue
                else:
                    break
            else:
                consecutive_empty_pages = 0
                page_reviews = []
                
                for i, block in enumerate(review_blocks):
                    try:
                        # 사용자 ID - 여러 선택자로 시도
                        user_id = ""
                        user_selectors = [
                            'span.info_item',
                            'span.badge_box',
                            '.user_info span',
                            '.reviewer_name',
                            'span[class*="info"]'
                        ]
                        
                        for selector in user_selectors:
                            try:
                                user_element = block.find_element(By.CSS_SELECTOR, selector)
                                user_id = user_element.text.strip()
                                if user_id:
                                    break
                            except:
                                continue
                    except:
                        user_id = ""
                    
                    try:
                        # 리뷰 내용 - 여러 선택자로 시도
                        review_text = ""
                        review_selectors = [
                            'div.comment_text',
                            '.review_content',
                            '.comment_content',
                            'div[class*="comment"]',
                            'div[class*="text"]'
                        ]
                        
                        for selector in review_selectors:
                            try:
                                review_element = block.find_element(By.CSS_SELECTOR, selector)
                                review_text = review_element.text.strip()
                                if review_text:
                                    break
                            except:
                                continue
                    except:
                        review_text = ""
                    
                    # 별점
                    rating = get_star_rating(block)
                    
                    # 유효한 리뷰만 추가 (사용자ID나 리뷰 내용이 있는 경우)
                    if user_id or review_text:
                        page_reviews.append({
                            'isbn': isbn,
                            'user_id': user_id,
                            'review': review_text,
                            'rating': rating,
                            'page': page
                        })
                
                reviews.extend(page_reviews)
                print(f"ISBN {isbn}: {page}페이지 - {len(page_reviews)}개 리뷰 수집 (누적: {len(reviews)}개)")
            
            # 다음 페이지로 이동
            if not go_to_next_page(driver, page):
                print(f"ISBN {isbn}: 더 이상 페이지가 없습니다.")
                break
                
            page += 1
        
        print(f"ISBN {isbn}: 총 {len(reviews)}개 리뷰 수집 완료")
        return reviews
        
    except Exception as e:
        print(f"ISBN {isbn}: 크롤링 중 오류 발생 - {str(e)}")
        import traceback
        traceback.print_exc()
        return []

def go_to_next_page(driver, current_page):
    """다음 페이지로 이동하는 함수"""
    try:
        # 다음 페이지 버튼 선택자들 - 제공된 정보와 일반적인 패턴 포함
        next_page_selectors = [
            f'//*[@id="ReviewList1"]/div[3]/div[2]/div/div[2]/div/a[{current_page + 1}]',  # 특정 페이지 번호
            'button.btn_page.next',  # 다음 버튼
            '//*[@id="ReviewList1"]/div[3]/div[2]/div/div[2]/button[2]',  # 제공된 정보
            '//button[contains(@class, "next")]',
            '//button[contains(@class, "btn_page") and contains(@class, "next")]',
            '//a[contains(@class, "next")]'
        ]
        
        for selector in next_page_selectors:
            try:
                if selector.startswith('//') or selector.startswith('//*'):
                    next_element = driver.find_element(By.XPATH, selector)
                else:
                    next_element = driver.find_element(By.CSS_SELECTOR, selector)
                
                # 버튼이 비활성화되어 있는지 확인
                if ("disabled" in next_element.get_attribute("class") or 
                    not next_element.is_enabled() or
                    "inactive" in next_element.get_attribute("class")):
                    continue
                
                # 버튼 클릭
                driver.execute_script("arguments[0].scrollIntoView(true);", next_element)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_element)
                
                # 페이지 로딩 대기
                time.sleep(3)
                return True
                
            except Exception as e:
                continue
        
        return False
        
    except Exception as e:
        print(f"다음 페이지 이동 중 오류 - {str(e)}")
        return False

def main():
    """메인 실행 함수"""
    # 기존 리뷰 파일에서 누락된 ISBN 리스트 자동 추출
    try:
        kb_review_best = pd.read_csv('[교보] 베스트셀러 리뷰.csv')
        # 중복 제거: drop_duplicates() 추가
        no_review_list = kb_review_best[kb_review_best['review'].isna()]['isbn'].dropna().astype(str).drop_duplicates().tolist()
        print(f"누락된 리뷰가 있는 고유 ISBN {len(no_review_list)}개를 발견했습니다.")
        print(f"누락 ISBN 목록: {no_review_list[:10]}{'...' if len(no_review_list) > 10 else ''}")
        isbn_list = no_review_list
    except FileNotFoundError:
        print("'[교보] 베스트셀러 리뷰.csv' 파일을 찾을 수 없습니다. 기본 ISBN 리스트를 사용합니다.")
        # 기본 ISBN 리스트 (파일이 없을 경우)
        isbn_list = [
            "9791168270749", 
            "9788901294742",
            # 여기에 직접 ISBN을 추가할 수도 있습니다
        ]
    except Exception as e:
        print(f"CSV 파일 읽기 중 오류 발생: {str(e)}")
        print("기본 ISBN 리스트를 사용합니다.")
        isbn_list = [
            "9791168270749", 
            "9788901294742",
        ]
    
    if not isbn_list:
        print("크롤링할 ISBN이 없습니다.")
        return
    
    driver = setup_driver()
    all_reviews = []
    
    try:
        for i, isbn in enumerate(isbn_list, 1):
            print(f"\n=== {i}/{len(isbn_list)} - ISBN: {isbn} 크롤링 시작 ===")
            
            reviews = crawl_reviews_for_book(driver, isbn)
            all_reviews.extend(reviews)
            
            print(f"ISBN {isbn} 완료. 현재까지 총 {len(all_reviews)}개 리뷰 수집")
            
            # 각 ISBN 처리 후 잠시 대기
            time.sleep(5)
        
        # 모든 리뷰를 하나의 DataFrame으로 합치기
        if all_reviews:
            df = pd.DataFrame(all_reviews)
            
            # 기존 파일이 있다면 업데이트, 없다면 새로 생성
            try:
                existing_df = pd.read_csv('[교보] 베스트셀러 리뷰.csv')
                
                # 새로 수집한 리뷰로 기존 데이터 업데이트
                # ISBN별로 기존 리뷰가 없는 것만 새로운 리뷰로 교체
                for isbn in df['isbn'].unique():
                    # 해당 ISBN의 기존 데이터 제거
                    existing_df = existing_df[existing_df['isbn'] != isbn]
                
                # 새로운 리뷰 데이터 추가
                updated_df = pd.concat([existing_df, df], ignore_index=True)
                
                # 기존 파일 백업
                import shutil
                shutil.copy('[교보] 베스트셀러 리뷰.csv', '[교보] 베스트셀러 리뷰_backup.csv')
                
                # 업데이트된 데이터 저장
                updated_df.to_csv('[교보] 베스트셀러 리뷰.csv', index=False, encoding='utf-8-sig')
                output_filename = '[교보] 베스트셀러 리뷰.csv (업데이트됨)'
                
                print(f"기존 파일을 업데이트했습니다. 백업 파일: [교보] 베스트셀러 리뷰_backup.csv")
                
            except FileNotFoundError:
                # 기존 파일이 없으면 새로 생성
                output_filename = f'book_reviews_missing_{len(isbn_list)}books.csv'
                df.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"\n=== 크롤링 완료 ===")
            print(f"총 도서 수: {len(isbn_list)}")
            print(f"총 리뷰 수: {len(df)}")
            print(f"파일명: {output_filename}")
            print(f"\n도서별 리뷰 수:")
            print(df.groupby('isbn').size().to_string())
            
            # 별점 통계
            if df['rating'].notna().sum() > 0:
                print(f"\n별점 통계:")
                print(f"평균 별점: {df['rating'].mean():.2f}")
                print(f"별점 분포:")
                print(df['rating'].value_counts().sort_index().to_string())
            
        else:
            print("수집된 리뷰가 없습니다.")
            
    except KeyboardInterrupt:
        print("\n사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

누락된 리뷰가 있는 고유 ISBN 1639개를 발견했습니다.
누락 ISBN 목록: ['9788956605418', '9788983945068', '9788937886966', '9788991075535', '9788936456085', '9791170610038', '9788965465072', '9791167521859', '9791191825145', '9788960519466']...

=== 1/1639 - ISBN: 9788956605418 크롤링 시작 ===
ISBN 9788956605418: 도서 상세 페이지로 이동 완료
ISBN 9788956605418: 리뷰 탭 클릭 성공
ISBN 9788956605418: 1페이지 - div.comment_item로 10개 리뷰 블록 발견
ISBN 9788956605418: 전체 리뷰 로딩 탭 클릭 시도
ISBN 9788956605418: 전체 리뷰 로딩 탭 클릭 성공
ISBN 9788956605418: 전체 리뷰 로딩 후 10개 리뷰 블록 발견
ISBN 9788956605418: 1페이지 - 10개 리뷰 수집 (누적: 10개)
ISBN 9788956605418: 2페이지 - div.comment_item로 10개 리뷰 블록 발견
ISBN 9788956605418: 2페이지 - 10개 리뷰 수집 (누적: 20개)
ISBN 9788956605418: 3페이지 - div.comment_item로 10개 리뷰 블록 발견
ISBN 9788956605418: 3페이지 - 10개 리뷰 수집 (누적: 30개)
ISBN 9788956605418: 4페이지 - div.comment_item로 10개 리뷰 블록 발견
ISBN 9788956605418: 4페이지 - 10개 리뷰 수집 (누적: 40개)
ISBN 9788956605418: 5페이지 - div.comment_item로 10개 리뷰 블록 발견
ISBN 9788956605418: 5페이지 - 10개 리뷰 수집 (누적: 50개)
ISBN 9788956605418: 6페이지